<a href="https://colab.research.google.com/github/nairsatish/4540/blob/master/CNNMNIST_BinaryClassif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
RunningInCOLAB = 'google.colab' in str(get_ipython())

if RunningInCOLAB:
    !pip install torch, torchvision

ERROR: Invalid requirement: 'torch,'


In [2]:
import torch
import torch.nn as nn
import torchvision
from torchvision.transforms import ToTensor

## CNN Details

The kernel/filter/weight matrix is a 5x5 matrix. Out channels corresponds to the number of created feature maps, and therefore impacts the number of kernels.

MaxPool2D is a 2x2 sliding window taking the maximum value in that window.

Since this is the MNIST dataset, each image is 28x28. If we take a batch size of 1, this means our structure is:

Input (28x28) **->**
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Conv2D (16x28x28) **->** ReLU (16x28x28) **->** MaxPool2D (16x14x14) **->**
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Conv2D (32x14x14) **->** RelU (32x14x14) **->** MaxPool2D (32x7x7) **->**
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Flatten (1568x1) **->** FCN **->**
Returned Output (10x1)

In [3]:
#Derived from https://medium.com/@nutanbhogendrasharma/pytorch-convolutional-neural-network-with-mnist-dataset-4e8a4265e118

class ExampleCNN(nn.Module):
    def __init__(self):
        super(ExampleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(5,5), stride=(1,1), padding=2)
        self.activation = nn.ReLU()
        self.pool2d = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(5,5), stride=(1,1), padding=2)
        self.fcn = nn.Linear(32 * 7 * 7, 10)
        
    def forward(self, x):
        x = self.pool2d(self.activation(self.conv1(x)))
        x = self.pool2d(self.activation(self.conv2(x)))
        
        x = x.view(x.size(0), -1)
        
        out = self.fcn(x)
        return out

In [4]:
from torch.utils.data import DataLoader

"""
Running in COLAB assumes you've not imported the entire repository and simply saves the data with the notebook

Running in Jupyter Notebook assumes you have imported the entire repository and stores it in the data folder
"""
if RunningInCOLAB:
    train_data = torchvision.datasets.MNIST('.', train=True, transform=ToTensor(), download=True)
    test_data = torchvision.datasets.MNIST('.', train=False, transform=ToTensor(), download=True)
else:
    train_data = torchvision.datasets.MNIST('.', train=True, transform=ToTensor(), download=True)
    test_data = torchvision.datasets.MNIST('.', train=False, transform=ToTensor(), download=True)
batch_size = 124
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
loaders = {'train' : train_loader, 'test' : test_loader}

100%|██████████| 9912422/9912422 [00:00<00:00, 79310570.20it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 9854038.38it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 27659140.12it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 3529836.72it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [ ]:
from torch.autograd import Variable
from torch import optim

cnn = ExampleCNN()


num_epochs = 10

loss_func = nn.CrossEntropyLoss()   
optimizer = optim.Adam(cnn.parameters(), lr = 0.01)   

def train(epochs, network, data_loaders):
    
    network.train()
        
    # Train the model
    total_step = len(data_loaders['train'])
        
    for epoch in range(epochs):
        for i, (x, y) in enumerate(data_loaders['train']):
            
            # gives batch data, normalize x when iterate train_loader
            b_x = Variable(x)   # batch x
            b_y = Variable(y)   # batch y
            b_y[b_y <= 4] = 0
            b_y[b_y > 4] = 1
            output = network(b_x)
            loss = loss_func(output, b_y)
            
            # clear gradients for this training step   
            optimizer.zero_grad()           
            
            # backpropagation, compute gradients 
            loss.backward()                # apply gradients             
            optimizer.step()                
            
            if (i+1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                       .format(epoch + 1, epochs, i + 1, total_step, loss.item()))

train(num_epochs, cnn, loaders)

Epoch [1/10], Step [100/484], Loss: 0.1933
Epoch [1/10], Step [200/484], Loss: 0.0376
Epoch [1/10], Step [300/484], Loss: 0.0622
Epoch [1/10], Step [400/484], Loss: 0.1148
Epoch [2/10], Step [100/484], Loss: 0.0352
Epoch [2/10], Step [200/484], Loss: 0.0396
Epoch [2/10], Step [300/484], Loss: 0.0150
Epoch [2/10], Step [400/484], Loss: 0.0337
Epoch [3/10], Step [100/484], Loss: 0.0161
Epoch [3/10], Step [200/484], Loss: 0.0279
Epoch [3/10], Step [300/484], Loss: 0.0197
Epoch [3/10], Step [400/484], Loss: 0.0552
Epoch [4/10], Step [100/484], Loss: 0.0335
Epoch [4/10], Step [200/484], Loss: 0.0593
Epoch [4/10], Step [300/484], Loss: 0.0124
Epoch [4/10], Step [400/484], Loss: 0.0491
Epoch [5/10], Step [100/484], Loss: 0.0211
Epoch [5/10], Step [200/484], Loss: 0.0263
Epoch [5/10], Step [300/484], Loss: 0.0324
Epoch [5/10], Step [400/484], Loss: 0.0452
Epoch [6/10], Step [100/484], Loss: 0.0877
Epoch [6/10], Step [200/484], Loss: 0.0525
Epoch [6/10], Step [300/484], Loss: 0.0360
Epoch [6/10

In [ ]:
def test():
    # Test the model
    cnn.eval()    
    
    with torch.no_grad():
        for x, y in loaders['test']:
            y[y <= 4] = 0
            y[y > 4] = 1
            out = cnn(x)
            y_prediction = torch.max(out, 1)[1].data.squeeze()
            accuracy = (y_prediction == y).sum().item() / float(y.size(0))
            pass
        print('Test Accuracy of the model on the 10000 test images: %.2f' % accuracy)
    
    pass
test()

In [ ]:
sample = next(iter(loaders['test']))
images, labels = sample
actual_number = labels[:10].numpy()
actual_number[actual_number <= 4] = 0
actual_number[actual_number > 4] = 1
test_output = cnn(images[:10])
y_predictions = torch.max(test_output, 1)[1].data.numpy().squeeze()
print(f'Prediction number: {y_predictions}')
print(f'Actual number: {actual_number}')

In [ ]:
print(cnn.conv1.weight)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

layer1_filter = 0

plt.imshow(np.squeeze(cnn.conv1.weight.detach().numpy()[layer1_filter,:,:,:]), cmap='gray')
plt.show()

In [ ]:
layer2_filter = 0

plt.imshow(cnn.conv2.weight.detach().numpy()[layer2_filter, layer1_filter, :, :], cmap='gray')
plt.show()